<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predict the best drug for heart treatment with IBM Watson Machine Learning (SPSS)</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/drug-selection/raw/master/images/heart_banner.png" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to load SPSS predictive model to Bluemix Cloud and start scoring new data. This notebook introduces commands for getting data, model persistance to Watson Machine Learning repository, model deployment, and batch scoring.

Some familiarity with Python is helpful. This notebook uses Python 2.0.

You will use published on git data set, **drug_batch_data**, which details anonymous patients records. Use the details of this data set to predict best drug for heart disease treatment.

## Learning goals

The learning goals of this notebook are:

-  Load a CSV file into DB2 Warehouse on Cloud Service on Bluemix.
-  Persist SPSS model in Watson Machine Learning repository.
-  Deploy a model for batch scoring using Wastson Machine Learning API.
-  Score sample scoring data using the Watson Machine Learning API.
-  Explore and visualize prediction result using the plotly package.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Persist model](#persistence)
3.	[Score in a Cloud](#scoring)
4.	[Explore predictions](#predictions)
4.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Create a [DB2 Warehouse on Cloud](https://console.bluemix.net/docs/services/Db2whc/index.html#getting_started) instance (an entry plan is offered). 
-  Upload **drugTrain2** data to Db2 Warehouse on Cloud.


### Create the DRUGTRAIN2 table in Db2 Warehouse on Cloud

1.  Download [drug_batch_data.csv](https://github.com/pmservice/drug-selection/blob/master/data/drug_batch_data.csv) file from git repository.
2.  Click the **Open the console to get started with Db2 Warehouse on Cloud** icon.
3.  Select the **Load Data** and **Desktop** load type.
4.  Drug and drop previously downloaded file and press **Next**
5.  Table **DRUG_BATCH_DATA** with uploaded data should be created for you.

<a id="persistence"></a>
## 2. Persist model

In this section you will learn how to store your model in Watson Machine Learning repository by using REST API.

**Action**: Put the credentials of your Watson Machine Learning service instance.

In [22]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

**Tip**: Credentials can be found on **Service Credentials** tab of service instance created in Bluemix.

### 2.1: Download sample SPSS stream

**Action**: Download sample SPSS stream from git project using wget command.</div>

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.<BR><BR>
!pip install wget --user <BR>

In [68]:
!wget https://github.com/pmservice/drug-selection/raw/master/model/Drug1n_capitalized.str

--2017-05-31 05:15:42--  https://github.com/pmservice/drug-selection/raw/master/model/Drug1n_capitalized.str
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pmservice/drug-selection/master/model/Drug1n_capitalized.str [following]
--2017-05-31 05:15:43--  https://raw.githubusercontent.com/pmservice/drug-selection/master/model/Drug1n_capitalized.str
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47368 (46K) [application/octet-stream]
Saving to: ‘Drug1n_capitalized.str’

100%[======================================>] 47,368      --.-K/s   in 0.002s  

2017-05-31 05:15:43 (18.3 MB/s) - ‘Drug1n_capitalized.str’ saved [47

**Tip**: if you are using your own stream make sure that columns names used in stream and one in database have the same capitalization (e.g: UPPER_CASE letters).

### 2.2: Deploy the Drug1n_capitalized.str to Watson Machine Learning service

In [38]:
import urllib3, requests, json

In [69]:
context_id = "drug_cap_stream"
upload_endpoint = wml_credentials['url'] + "/pm/v1/file/" + context_id + "?accesskey=" + wml_credentials['access_key']
files = {'file': ('Drug1n_capitalized.str', open('Drug1n_capitalized.str', 'rb'))}

In [70]:
upload_response = requests.put(upload_endpoint, files=files)

print upload_response
print upload_response.text

<Response [200]>
https://palbyp.pmservice.ibmcloud.com/pm/v1/file/drug_cap_stream


As you can see model is deployed successfuly to Watson Machine Learning service on Cloud.

**Tip**: *context_id* can be any string that describes your model

<a id="scoring"></a>
## 3. Score in a Cloud using batch job

In this section you will learn how to create batch job and score records present in Db2 Warehouse on Cloud using the Watson Machine Learning REST API. 
For more information about REST APIs, see the [Bluemix Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html).

### 3.1: Create connection map to Db2 Warehouse on Cloud table with data

Using Db2 Warehouse on Cloud credentials information from Bluemix update **host**,**port**,**db**,**username** and **password** values in dbDefinitions dictionary below.

In [71]:
dbDefinitions = {
    "db1":{
         "type":"DashDB",
         "host":"awh-yp-small02.services.dal.bluemix.net",
         "port":50000,
         "db":"BLUDB",
         "username":"***",
         "password":"***"
      }
   }

**Tip**: all required fields can be found on **Service Credentials** tab of Db2 Warehouse on Cloud service instance created in Bluemix.

Update **table** name (if you used different than DRUG_BATCH_DATA) and **node** name to reflect you model's input/output node name in below dictionary. 
You can also update result **table** name from RESULTS to any custom string in exports section.

In [72]:
settings = {
      "inputs":[
         {
            "odbc":{
               "dbRef":"db1",
               "table":"DRUG_BATCH_DATA"
            },
            "node":"scoreInput",
            "attributes":[

            ]
         }
      ],
      "exports":[
         {
            "odbc":{
               "dbRef":"db1",
               "table":"RESULTS_DRUG",
               "insertMode":"Create"
            },
            "node":"Table",
            "attributes":[

            ]
         }
      ]
   }

**Tip**: database table name with data to score should be put into **inputs: table**; **inputs/exports: node** is the name of input and output node used in modeler stream

### 3.2: Submit batch job

In [73]:
job_id = "drug_job5"
batch_endpoint = wml_credentials['url'] + "/pm/v1/jobs/" + job_id + "?accesskey=" + wml_credentials['access_key']

batch_payload = {
    "action":"BATCH_SCORE", 
    "model":{
      "id":"drug_cap_stream",
      "name":"Drug1n_capitalized.str"
   },
   "dbDefinitions": dbDefinitions,
   "setting": settings
}

batch_header = {"Content-Type": "application/json"}

In [74]:
batch_response = requests.post(batch_endpoint, json=batch_payload, headers=batch_header)

print batch_response
print batch_response.text

<Response [200]>
{"namespace":"us-south$634640a0-1f9f-4df8-8ea6-95ca7326df20","storageId":"us-south$634640a0-1f9f-4df8-8ea6-95ca7326df20/drug_job5","action":"BATCH_SCORE","createdAt":1496225893369,"creater":"Tenant [instanceId=634640a0-1f9f-4df8-8ea6-95ca7326df20, planId=3f6acf43-ede8-413a-ac69-f8af3bb0cbfe, appId=null, enable=true]","id":"drug_job5","model":{"id":"drug_cap_stream","name":"Drug1n_capitalized.str"},"status":"PENDING","setting":{"exports":[{"odbc":{"insertMode":"Create","dbRef":"db1","table":"RESULTS_DRUG"},"node":"Table","attributes":[]}],"inputs":[{"odbc":{"dbRef":"db1","table":"DRUG_BATCH_DATA"},"node":"scoreInput","attributes":[]}],"parameterOverride":[]},"dbDefinitions":{"db1":{"db":"BLUDB","host":"awh-yp-small02.services.dal.bluemix.net","password":"n2#BP@mwW6Dl","port":50000,"username":"dash111858","type":"DashDB"}}}


Batch job has been submitted. Now, you can check the status of your batch job using the following REST API method.

In [76]:
batch_status_response = requests.get(batch_endpoint)

print batch_status_response
print batch_status_response.text

<Response [200]>
{"result":{"jobId":"drug_job5","jobStatus":"SUCCESS"},"namespace":"us-south$634640a0-1f9f-4df8-8ea6-95ca7326df20","storageId":"us-south$634640a0-1f9f-4df8-8ea6-95ca7326df20/drug_job5","action":"BATCH_SCORE","createdAt":1496225893369,"creater":"Tenant [instanceId=634640a0-1f9f-4df8-8ea6-95ca7326df20, planId=3f6acf43-ede8-413a-ac69-f8af3bb0cbfe, appId=null, enable=true]","finishedAt":1496225918066,"id":"drug_job5","model":{"id":"drug_cap_stream","name":"Drug1n_capitalized.str"},"status":"SUCCESS","setting":{"exports":[{"odbc":{"insertMode":"Create","dbRef":"db1","table":"RESULTS_DRUG"},"node":"Table","attributes":[]}],"inputs":[{"odbc":{"dbRef":"db1","table":"DRUG_BATCH_DATA"},"node":"scoreInput","attributes":[]}],"parameterOverride":[]},"dbDefinitions":{"db1":{"db":"BLUDB","host":"awh-yp-small02.services.dal.bluemix.net","password":"n2#BP@mwW6Dl","port":50000,"username":"dash111858","type":"DashDB"}}}


As we can see our batch job status is SUCCESS. Prediction results are store in table RESULTS_DRUG. Let's connect to Db2 Warehouse on Cloud and explore it.

## 4. Explore predictions 

In this section we will connect to RESULTS_DRUG table using Spark read method and explore prediction results.

### 4.1: Data exploration

Using below code you can read predictions results into Spark Data Frame.

In [ ]:
db2_credentials = {
    'jdbcurl': 'jdbc:db2://awh-yp-small02.services.dal.bluemix.net:50000/BLUDB',
    'user': '***',
    'password': '***'
}

In [ ]:
db2_properties = {x: db2_credentials.get(x) for x in db2_credentials.keys() if x in ['jdbcurl', 'user', 'password']}
tablename = "{schema}.{table}".format(schema=db2_credentials['user'], table='RESULTS_DRUG')

In [80]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
data = spark.read.jdbc(db2_properties['jdbcurl'], table=tablename, properties=db2_properties)
data.head()

Row(AGE=23, SEX=u'F', BP=u'HIGH', CHOLESTEROL=u'HIGH', NA=0.792535, K=0.031258, $N-DRUG=u'drugY', $NC-DRUG=0.9999929901781156)

As you can see data has been loaded correctly. Now we can check the schema of prediction data using printSchema() method.

In [78]:
data.printSchema()

root
 |-- AGE: long (nullable = true)
 |-- SEX: string (nullable = true)
 |-- BP: string (nullable = true)
 |-- CHOLESTEROL: string (nullable = true)
 |-- NA: double (nullable = true)
 |-- K: double (nullable = true)
 |-- $N-DRUG: string (nullable = true)
 |-- $NC-DRUG: double (nullable = true)



Now you can call show() method to preview prediction data.

In [79]:
data.show()

+---+---+------+-----------+--------+--------+-------+------------------+
|AGE|SEX|    BP|CHOLESTEROL|      NA|       K|$N-DRUG|          $NC-DRUG|
+---+---+------+-----------+--------+--------+-------+------------------+
| 23|  F|  HIGH|       HIGH|0.792535|0.031258|  drugY|0.9999929901781156|
| 47|  M|   LOW|       HIGH|0.739309|0.056468|  drugC|0.9745257637443014|
| 47|  M|   LOW|       HIGH|0.697269|0.068944|  drugC|0.9991828775434979|
| 28|  F|NORMAL|       HIGH|0.563682|0.072289|  drugX|0.9956213726883215|
| 61|  F|   LOW|       HIGH|0.559294|0.030998|  drugY|0.9997859875220865|
| 22|  F|NORMAL|       HIGH|0.676901|0.078647|  drugX|0.9943009058967174|
| 49|  F|NORMAL|       HIGH|0.789637|0.048518|  drugY|0.9994412857017156|
| 41|  M|   LOW|       HIGH|0.766635|0.069461|  drugC|0.9992996209099003|
| 60|  M|NORMAL|       HIGH|0.777205| 0.05123|  drugY|0.9961953878310243|
| 43|  M|   LOW|     NORMAL|0.526102|0.027164|  drugY|0.9909220997143252|
| 47|  F|   LOW|       HIGH|0.896056|0

As you can see two columns with predicted drug (N-DRUG) and probability (NC-DRUG) are shown.

In [83]:
data.select("$N-DRUG").groupBy("$N-DRUG").count().show()

+-------+-----+
|$N-DRUG|count|
+-------+-----+
|  drugC|    5|
|  drugA|    1|
|  drugY|   12|
|  drugX|    4|
+-------+-----+



You can easily calculate drugs distribution using select statement.

### 4.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Example**:  First, you need to install required packages. You can do it by running the following code. Run it only one time.

!pip install plotly --user

!pip install cufflinks --user

In [ ]:
!pip install plotly --user 
!pip install cufflinks --user

Import Plotly and other required packages.

In [85]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go

init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Convert the Apache Spark DataFrame to a Pandas DataFrame.

In [86]:
data_pdf = data.toPandas()

Plot a pie chart that shows drugs distribution.

In [90]:
cumulative_stats = data_pdf.groupby(['$N-DRUG']).count()

drug_data = [go.Pie(
            labels=cumulative_stats.index,
            values=cumulative_stats['$NC-DRUG'],
    )]

drug_layout = go.Layout(
    title='Heart treatment drugs distribution',
)

fig = go.Figure(data=drug_data, layout=drug_layout)
iplot(fig)

With this data set, you might want to do some analysis of the mean K value per drug type by using a bar chart.

In [96]:
age_data = [go.Bar(
            y=data_pdf.groupby(['$N-DRUG']).mean()["K"],
            x=cumulative_stats.index
            
    )]

age_layout = go.Layout(
    title='Mean K per recommended drug',
    xaxis=dict(
        title = "Drug",
        showline=False,),
    yaxis=dict(
        title = "Mean K",
        ),
)

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The drugC and drugX is recommended for patients with high value of K.

<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](www.ibm.com)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.